# Train a Hypergraph Neural Network

In this notebook, we will create and train a two-step message passing network in the hypergraph domain. We will use a benchmark dataset, MUTAG (from the TUDataset), to train the model to perform binary classification at the level of the h. 

In [1]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

from torch_geometric.datasets import TUDataset
from torch_geometric.utils.convert import to_networkx
from toponetx import SimplicialComplex
from topomodelx.nn.hypergraph.template_layer import TemplateLayer

# Pre-processing

## Import data ##

The first step is to import the dataset, MUTAG, a benchmark dataset for graph classification. We then lift each graph into our domain of choice, a hypergraph.

We will also retrieve:
- input signal on the edges for each of these hypergraphs, as that will be what we feed the model in input
- the binary label associated to the hypergraph

In [3]:
dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG", use_edge_attr=True)
dataset = dataset[:20]
hg_list = []
x_1_list = []
y_list = []
for graph in dataset:
    hg = SimplicialComplex(to_networkx(graph)).to_hypergraph()
    hg_list.append(hg)
    x_1 = torch.chunk(graph.edge_attr, 2, dim=0)[1]
    x_1_list.append(x_1)
    y_list.append(int(graph.y))

## Define neighborhood structures. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on each simplicial complex. In the case of this architecture, we need the boundary matrix (or incidence matrix) $B_1$ with shape $n_\text{nodes} \times n_\text{edges}$.

In [4]:
incidence_1_list = []
for hg in hg_list:
    incidence_1 = hg.incidence_matrix()
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    incidence_1_list.append(incidence_1)

# Create the Neural Network

Using the TemplateLayer class, we create a neural network with stacked layers.

In [5]:
channels_edge = x_1_list[0].shape[1]
channels_node = dataset[0].x.shape[1]

In [6]:
class TemplateNN(torch.nn.Module):
    """Neural network implementation of Template for hypergraph classification.

    Parameters
    ---------
    channels_edge : int
        Dimension of edge features
    channels_node : int
        Dimension of node features
    n_layer : 2
        Amount of message passing layers.

    """

    def __init__(self, channels_edge, channels_node, n_layers=2):
        super().__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(
                TemplateLayer(
                    in_channels=channels_edge,
                    intermediate_channels=channels_node,
                    out_channels=channels_edge,
                )
            )
        self.layers = torch.nn.ModuleList(layers)
        self.linear = torch.nn.Linear(channels_edge, 1)

    def forward(self, x_1, incidence_1):
        """Forward computation through layers, then linear layer, then global max pooling.

        Parameters
        ---------
        x_1 : tensor
            shape = [n_edges, channels_edge]
            Edge features.

        incidence_1 : tensor
            shape = [n_nodes, n_edges]
            Boundary matrix of rank 1.

        Returns
        --------
        _ : tensor
            shape = [1]
            Label assigned to whole complex.
        """
        for layer in self.layers:
            x_1 = layer(x_1, incidence_1)
        pooled_x = torch.max(x_1, dim=0)[0]
        return torch.sigmoid(self.linear(pooled_x))

# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [7]:
model = TemplateNN(channels_edge, channels_node, n_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.5)
crit = torch.nn.CrossEntropyLoss()

Split the dataset into train and test sets.

In [8]:
x_1_train, x_1_test = train_test_split(x_1_list, test_size=0.2, shuffle=False)
incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=0.2, shuffle=False
)
y_train, y_test = train_test_split(y_list, test_size=0.2, shuffle=False)

The following cell performs the training, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [9]:
test_interval = 2
num_epochs = 5
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    optimizer.zero_grad()
    correct = 0
    num_samples = 0
    for x_1, incidence_1, y in zip(x_1_train, incidence_1_train, y_train):
        y_hat = model(x_1.float(), incidence_1.float())
        y = torch.tensor(y).unsqueeze(0)
        loss = crit(y_hat.float(), y.float())
        correct += (y_hat.argmax() == y).sum().item()
        num_samples += 1
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
    train_acc = correct / num_samples
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {train_acc:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            num_samples = 0
            correct = 0
            for x_1, incidence_1, y in zip(x_1_test, incidence_1_test, y_test):
                y = torch.tensor(y).long()
                y_hat = model(x_1.float(), incidence_1.float())
                correct += (y_hat.argmax() == y).sum().item()
                num_samples += 1
            test_acc = correct / num_samples
            print(f"Test_acc: {test_acc:.4f}", flush=True)

Epoch: 1 loss: 0.0000 Train_acc: 0.3750
Epoch: 2 loss: 0.0000 Train_acc: 0.3750
Test_acc: 0.5000
Epoch: 3 loss: 0.0000 Train_acc: 0.3750
Epoch: 4 loss: 0.0000 Train_acc: 0.3750
Test_acc: 0.5000
Epoch: 5 loss: 0.0000 Train_acc: 0.3750
